## FFD_for_conflict関数の引数にbin_dictとbins_group_usingを含んでないことがベータ版である理由

## 定数定義
$$
V=\{1,\ldots,n\}:アイテムの集合\\
B=\{1,\ldots,n\}:ビンの集合\\
M=\{1,\ldots,m\}:グループの集合\\
w_i:アイテムiの大きさ\\
z_{il}\in\{0,1\}:アイテムiのグループlへの所属を表すバイナリ定数\\
r_l:グループlがビンを1つ使用するのにかかるコスト\\
C:ビンの容量\\
E=\{(s,t) ~|~ w_s+w_t > C~または~sとtは同じビンに割り当てられられない\}:枝の集合\\
G=(V,E):~VとEから成るグラフ\\
$$

## 変数定義
$$
x_{ik}\in\{0,1\}:アイテムiのビンkへの割り当てを表すバイナリ変数\\
y_k\in\{0,1\}:ビンkの使用を表すバイナリ変数\\
b_{kl}\in\{0,1\}:グループlのビンkの使用を表すバイナリ変数\\
$$

## 目的関数
$$
\begin{align}
\text{minimize} & \displaystyle\sum_{k=1}^n y_k + \sum_{k=1}^n \sum_{l=1}^m r_l b_{kl} - \sum_{l=1}^m r_l
\end{align}
$$

## 制約式
$$
\begin{align}
\text{subject to}
\displaystyle\sum_{i=1}^n w_i x_{ik} \leq C y_k&~~~~~(k=1,\dots,n)\\
\displaystyle\sum_{k=1}^n x_{ik} = 1&~~~~~(i=1,\dots,n)\\
x_{sk}+x_{tk} \leq 1&~~~~~((s,t)\in E,~k=1,\dots,n)\\
b_{kl} \geq x_{jk} z_{jl}&~~~~~(k=1,\dots,n,~l=1,\dots,m,~j=1,\dots,n)\\
x_{ik}\in\{0,1\}&~~~~~(i,k=1,\dots,n)\\
y_k\in\{0,1\}&~~~~~(k=1,\dots,n)\\
b_{kl}\in\{0,1\}&~~~~~(k=1,\dots,n,~l=1,\dots,m)\\
\end{align}
$$

# ファイルの読み込み 

In [31]:
filename = "C125.9.clq.txt"
E=[]
with open(filename,mode="r") as f:
    lines = f.readlines()
    for i,v in enumerate(lines):
        line = v
        if "number of vertices" in line:
            n = int(line.split(" ")[-1])
        if line[0]=="e":
            start = i
            break
E = lines[start:-1] #ファイル名とEOFを除く
for i in range(len(E)):
    #E[i] = tuple(map(int,E[i][2:-1].split(" "))) #数字の数え方が１から始まってしまう
    temp = E[i][2:-1].split(" ")
    E[i] = (int(temp[0])-1,int(temp[1])-1)

# 問題設定

## 基本設定

In [32]:
import random
import numpy as np
random.seed(1)

#アイテムの個数
n

#アイテムの大きさ
w=[]
maximum=0
for i in range(n):
    w.append(random.randint(0,maximum))
    
#ビンの容量
C={b:maximum+1 for b in range(n)}

#枝
E
    
#コンフリクト
conf={}
n_conf = len(E)
for i in range(n_conf):
    temp = [0]*n
    temp[E[i][0]-1] = 1
    temp[E[i][1]-1] = 1
    conf[f"conf{i}"] = temp
    

#グループ
G=[]#[tuple(np.arange(1,126))]

#グループを分けるコスト
r = [1]*len(G)

#ビンを使うコスト
bin_cost = 100

## グループ分け

In [33]:
N = np.arange(0,n) #アイテムのグループ分けの際にのみ使う定数
G  = []

random_list = list(np.random.choice(N,10))

index=0
while True:
    if random_list[index]>=len(N): #random[index]の値がNの個数より多かったらGにNをいれて終わり
        G.append(tuple(N))
        break
    group = random.sample(N.tolist(),random_list[index])
    N = np.array(list(set(N)-set(group)))
    G.append(tuple(group))
    index+=1

## コンフリクト辞書とノットコンフリクト辞書の作成

### コンフリクトがあるノード間に枝があるような辞書

In [34]:
N = np.arange(0,n) #辞書をつくる際にのみ使う定数
E_dict={}
notconf_E_dict={}
flag=E[0][0]
for i in range(len(E)):
    if flag!=E[i][0]:
        flag=E[i][0]
    E_dict[E[i][0]] = E_dict.setdefault(E[i][0],[]) + [E[i][1]]
    E_dict[E[i][1]] = E_dict.setdefault(E[i][1],[]) + [E[i][0]]
    notconf_E_dict[E[i][0]] = list(set(N)-set(E_dict[E[i][0]])-{E[i][0]})
    notconf_E_dict[E[i][1]] = list(set(N)-set(E_dict[E[i][1]])-{E[i][1]})

### コンフリクトがないノード間に枝があるような辞書

In [35]:
notconf_E = []
for i in notconf_E_dict:
    for j in notconf_E_dict[i]:
               notconf_E.append((i,j))

# FFD 

In [36]:
def capacity(w_i,bin_b,w,C): #bin_bの容量がw_iの大きさよりも大きいかを判定
    if (C-sum(w[i] for i in bin_b)) >= w_i:
        return True
    else:
        return False

In [37]:
def conflict(item,bin_b,E_dict): #bin_bに割り当てられているアイテムたちとw_iの間にコンフリクトがあるかを判定
    if item not in E_dict:
        return False
    for i in bin_b:
        if i in E_dict[item]:
                return False
    return True

In [38]:
def group(i,G): #アイテムiの属するグループの番号を返す関数
    for index,menber in enumerate(G):
        if i in menber:
            return index

# グループを並べ替える関数
## 1.グループのメンバーが多い順

## 2.コンフリクトの数の平均が多い順

## 3.コンフリクトの数の最高値が高い順

In [39]:
for i in G:
    print(i)

(89, 41, 63, 60, 14, 3, 39, 49, 43, 53, 101, 24, 33, 13, 32, 93, 65, 26, 77, 55, 104, 2, 28, 103, 50, 18, 4, 92, 20, 57, 90, 64, 86, 54, 69, 102, 80, 66, 95, 124, 67, 83, 119, 100, 73, 123, 91, 7, 38, 16, 27, 6, 118, 9, 71, 72, 76, 96, 115, 110, 75, 1, 35, 56, 78, 114, 37, 52, 111, 36, 29, 10, 97, 117, 45, 94, 108, 59, 113, 12, 22, 88, 61, 105, 74, 58, 81, 31, 116, 46, 99, 48, 51, 122, 87, 19, 40, 68, 70, 0, 5, 79, 30, 34)
(8, 11, 15, 17, 21, 23, 25, 42, 44, 47, 62, 82, 84, 85, 98, 106, 107, 109, 112, 120, 121)


In [40]:
sort_G_1 = sorted(G,key=lambda x:len(x),reverse=True)
print(sort_G_1)

[(89, 41, 63, 60, 14, 3, 39, 49, 43, 53, 101, 24, 33, 13, 32, 93, 65, 26, 77, 55, 104, 2, 28, 103, 50, 18, 4, 92, 20, 57, 90, 64, 86, 54, 69, 102, 80, 66, 95, 124, 67, 83, 119, 100, 73, 123, 91, 7, 38, 16, 27, 6, 118, 9, 71, 72, 76, 96, 115, 110, 75, 1, 35, 56, 78, 114, 37, 52, 111, 36, 29, 10, 97, 117, 45, 94, 108, 59, 113, 12, 22, 88, 61, 105, 74, 58, 81, 31, 116, 46, 99, 48, 51, 122, 87, 19, 40, 68, 70, 0, 5, 79, 30, 34), (8, 11, 15, 17, 21, 23, 25, 42, 44, 47, 62, 82, 84, 85, 98, 106, 107, 109, 112, 120, 121)]


In [41]:
#上の確め算
for group_number,x in enumerate(G):
    print(group_number,"\t",x[:3],"...","\t",len(x))

0 	 (89, 41, 63) ... 	 104
1 	 (8, 11, 15) ... 	 21


In [42]:
sort_G_2 = sorted(G,key=lambda x:sum(len(E_dict[i]) for i in x)/len(x),reverse=True)
print(sort_G_2)

[(89, 41, 63, 60, 14, 3, 39, 49, 43, 53, 101, 24, 33, 13, 32, 93, 65, 26, 77, 55, 104, 2, 28, 103, 50, 18, 4, 92, 20, 57, 90, 64, 86, 54, 69, 102, 80, 66, 95, 124, 67, 83, 119, 100, 73, 123, 91, 7, 38, 16, 27, 6, 118, 9, 71, 72, 76, 96, 115, 110, 75, 1, 35, 56, 78, 114, 37, 52, 111, 36, 29, 10, 97, 117, 45, 94, 108, 59, 113, 12, 22, 88, 61, 105, 74, 58, 81, 31, 116, 46, 99, 48, 51, 122, 87, 19, 40, 68, 70, 0, 5, 79, 30, 34), (8, 11, 15, 17, 21, 23, 25, 42, 44, 47, 62, 82, 84, 85, 98, 106, 107, 109, 112, 120, 121)]


In [43]:
#上の確め算
for group_number,x in enumerate(G):
    print(group_number,"\t",x[:3],"...","\t",sum(len(E_dict[i]) for i in x)/len(x))

0 	 (89, 41, 63) ... 	 111.4326923076923
1 	 (8, 11, 15) ... 	 111.28571428571429


In [44]:
sort_G_3 = sorted(G,key=lambda x:max(len(E_dict[i]) for i in x),reverse=True)
print(sort_G_2)

[(89, 41, 63, 60, 14, 3, 39, 49, 43, 53, 101, 24, 33, 13, 32, 93, 65, 26, 77, 55, 104, 2, 28, 103, 50, 18, 4, 92, 20, 57, 90, 64, 86, 54, 69, 102, 80, 66, 95, 124, 67, 83, 119, 100, 73, 123, 91, 7, 38, 16, 27, 6, 118, 9, 71, 72, 76, 96, 115, 110, 75, 1, 35, 56, 78, 114, 37, 52, 111, 36, 29, 10, 97, 117, 45, 94, 108, 59, 113, 12, 22, 88, 61, 105, 74, 58, 81, 31, 116, 46, 99, 48, 51, 122, 87, 19, 40, 68, 70, 0, 5, 79, 30, 34), (8, 11, 15, 17, 21, 23, 25, 42, 44, 47, 62, 82, 84, 85, 98, 106, 107, 109, 112, 120, 121)]


In [45]:
#上の確め算
for group_number,x in enumerate(G):
    for i in x:
        if len(E_dict[i])==max(len(E_dict[j]) for j in x):
            print(group_number,"\t",x[:3],"...","\t",i)

0 	 (89, 41, 63) ... 	 53
0 	 (89, 41, 63) ... 	 59
0 	 (89, 41, 63) ... 	 113
1 	 (8, 11, 15) ... 	 44


In [46]:
#グループを並べ替える関数_1
def sort_G_1(G):
    return sorted(G,key=lambda x:len(x),reverse=True)

In [47]:
#グループを並べ替える関数_2
def sort_G_2(G):
    return sorted(G,key=lambda x:sum(len(E_dict[i]) for i in x)/len(x),reverse=True)

In [48]:
#グループを並べ替える関数_3
def sort_G_3(G):
    return sorted(G,key=lambda x:max(len(E_dict[i]) for i in x),reverse=True)

In [20]:
def FFD_for_conflict(Item,w,C,E_dict,G): #コンフリクトの数が多い順にアイテムをビンに割り当てる
    bin_dict = {b:[] for b in range(len(Item))}
    bins_group_using = {g:-1 for g in range(len(G))}
    Item = sorted(E_dict,key=lambda x:len(E_dict[x]),reverse=True) #コンフリクトの数が多い順にアイテムを走査
    
    for i in Item:
        group_no = group(i,G)
        for b in range(len(bin_dict)): #インデックスが小さい順にビンを走査
            if capacity(w[i],bin_dict[b],w,C[b]) and conflict(i,bin_dict[b],E_dict): #容量制約を満たす　and　コンフリクトがない
                bin_dict[b] = bin_dict.setdefault(b,[]) + [i] #ビンbにアイテムiを入れる
                C[b] = C[b]-w[i] #ビンbのキャパシティーをアイテムiの大きさ分減少させる
                bins_group_using[group_no] += 1 #グループgroup_noの使用するビンの数を1増やす
                break  
                
    return bin_dict,bins_group_using

In [71]:
def FFD_for_conflict(Item,w,C,E_dict,G,bin_dict,bins_group_using): #コンフリクトの数が多い順にアイテムをビンに割り当てる
    """print("グループ",group(Item[0],G),"\t",len(Item),"個","\t",Item)
    print()"""
    Item = sorted(Item,key=lambda x:len(E_dict[x]),reverse=True) #コンフリクトの数が多い順にアイテムを走査
    
    for i in Item:
        group_no = group(i,G)
        for b in range(len(bin_dict)): #インデックスが小さい順にビンを走査
            if capacity(w[i],bin_dict[b],w,C[b]) and conflict(i,bin_dict[b],E_dict): #容量制約を満たす　and　コンフリクトがない
                bin_dict[b] = bin_dict.setdefault(b,[]) + [i] #ビンbにアイテムiを入れる
                C[b] = C[b]-w[i] #ビンbのキャパシティーをアイテムiの大きさ分減少させる
                bins_group_using[group_no] += 1 #グループgroup_noの使用するビンの数を1増やす
                break  
                
    return bin_dict,bins_group_using,C

In [72]:
def FFD_for_conflict_group(Item,w,C,E_dict,G):
    bin_dict = {b:[] for b in range(len(Item))}
    bins_group_using = {g:-1 for g in range(len(G))}
    
    G = sort_G_3(G) #グループの並べ替え
    for g in G: #グループごとにFFD_for_conflict()
        #print(len(g),g)
        bin_dict,bins_group_using,C = FFD_for_conflict(g,w,C,E_dict,G,bin_dict,bins_group_using)
    return bin_dict,bins_group_using

In [73]:
Item = list(range(n))
C={b:maximum+1 for b in range(n)}
bin_dict = {b:[] for b in range(len(Item))}
bins_group_using = {g:-1 for g in range(len(G))}

In [75]:
bin_dict,bins_group_using,C = FFD_for_conflict(Item,w,C,E_dict,G,bin_dict,bins_group_using)

In [76]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [53, 9, 53, 9]
1 [59, 18, 59, 18]
2 [113, 23, 113, 23]
3 [44, 110, 44, 110]
4 [103, 100, 103, 100]
5 [6, 29, 107, 6, 29, 107]
6 [7, 109, 7, 109]
7 [79, 91, 79, 91]
8 [98, 46, 81, 98, 46, 81]
9 [10, 58, 10, 58]
10 [28, 45, 28, 45]
11 [39, 73, 15, 39, 73, 15]
12 [48, 40, 48, 40]
13 [66, 0, 86, 66, 0, 86]
14 [97, 122, 97, 122]
15 [124, 5, 82, 124, 5, 82]
16 [17, 64, 17, 64]
17 [38, 76, 38, 76]
18 [69, 19, 106, 69, 19, 106]
19 [21, 3, 30, 21, 3, 30]
20 [33, 57, 33, 57]
21 [1, 12, 1, 12]
22 [8, 68, 115, 8, 68, 115]
23 [24, 34, 24, 34]
24 [47, 51, 47, 51]
25 [84, 108, 35, 84, 108, 35]
26 [85, 116, 72, 85, 116, 72]
27 [95, 118, 114, 95, 118, 114]
28 [4, 22, 94, 4, 22, 94]
29 [43, 56, 26, 43, 56, 26]
30 [71, 25, 71, 25]
31 [78, 117, 74, 78, 117, 74]
32 [90, 52, 90, 52]
33 [36, 42, 36, 42]
34 [61, 31, 67, 61, 31, 67]
35 [65, 55, 65, 55]
36 [83, 121, 89, 83, 121, 89]
37 [105, 120, 105, 120]
38 [27, 13, 63, 27, 13, 63]
39 [60, 119, 60, 119]
40 [70, 20, 70, 20]
41 [92, 49, 92, 49]
42 [123, 32, 1

In [77]:
print(bins_group_using)

{0: 207, 1: 41}


In [78]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

56


# FFD for conflict and group

In [64]:
Item = list(range(n))
C={b:maximum+1 for b in range(n)}

In [65]:
bin_dict,bins_group_using = FFD_for_conflict_group(Item,w,C,E_dict,G)

In [66]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [53, 9]
1 [59, 18]
2 [113, 5]
3 [103, 100]
4 [110, 64]
5 [7, 39]
6 [6, 29, 107]
7 [79, 91]
8 [28, 45]
9 [66, 114, 93]
10 [124, 40]
11 [10, 58]
12 [97, 122]
13 [48, 56]
14 [69, 83, 106]
15 [38, 76]
16 [46, 81, 98]
17 [33, 3]
18 [24, 34]
19 [95, 118, 14]
20 [73, 4, 26]
21 [1, 12]
22 [51, 71, 25]
23 [43, 108]
24 [57, 123]
25 [90, 52]
26 [78, 117, 74]
27 [116, 61, 31]
28 [68, 22, 99]
29 [0, 36, 94]
30 [30, 86]
31 [65, 60]
32 [105, 55]
33 [19, 112]
34 [92, 13, 63]
35 [119, 101, 50]
36 [27, 37, 111]
37 [70, 20]
38 [77, 104, 67]
39 [54, 80]
40 [88, 49, 75]
41 [32, 16, 115]
42 [102, 35]
43 [2, 41, 82]
44 [96, 89]
45 [87, 8, 11]
46 [72, 85]
47 [44]
48 [109]
49 [17]
50 [23]
51 [21, 120]
52 [47]
53 [84]
54 [42]
55 [121]
56 [62]
57 [15]
58 []
59 []
60 []
61 []
62 []
63 []
64 []
65 []
66 []
67 []
68 []
69 []
70 []
71 []
72 []
73 []
74 []
75 []
76 []
77 []
78 []
79 []
80 []
81 []
82 []
83 []
84 []
85 []
86 []
87 []
88 []
89 []
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 

In [67]:
w[39]+w[74]#+w[96]

0

In [68]:
print(bins_group_using)

{0: 103, 1: 20}


In [69]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

58
